In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
df=pd.read_csv('../data/le_train.csv',index_col=0)

In [2]:
trainX=df.drop('readmitted',axis=1).copy()
trainY=df['readmitted']

In [3]:
para= {"learning_rate" : [0.01,0.1,1],
       "n_estimators": [200,500,1000,2000,3000,4000],
       "max_depth" : [3,4,5,6],
       "subsample" : [0.8,0.9,1],
       "colsample_bytree" : [0.8,0.9,1]}

In [4]:
xgboost = xgb.XGBRegressor()
n_iter_search = 2
ran_search_xgb=RandomizedSearchCV(estimator=xgboost, param_distributions=para,n_iter=n_iter_search,
                                  scoring='neg_mean_squared_error', cv=5, return_train_score=True)

In [35]:
#xgboost = xgb.XGBClassifier()
#n_iter_search =2
#ran_search_xgb=RandomizedSearchCV(estimator=xgboost, param_distributions=para,n_iter=n_iter_search,
#                                  scoring='roc_auc', cv=5, return_train_score=True)

In [ ]:
# training_start = time.perf_counter()
# knn.fit(X_train_new, y_train_new)
# training_end = time.perf_counter()
# prediction_start = time.perf_counter()
# preds = knn.predict(X_test)
# prediction_end = time.perf_counter()
# acc_knn = (preds == y_test).sum().astype(float) / len(preds)*100    
# acc = accuracy_score(y_test, preds)
# pre = precision_score(y_test, preds)
# rec = recall_score(y_test, preds)
# AUC = roc_auc_score(y_test, preds)
# knn_train_time = training_end-training_start
# knn_prediction_time = prediction_end-prediction_start
                    
# print("Accuracy: {:.4%}".format(acc))
# print("Precision: {:.4%}".format(pre))
# print("Recall: {:.4%}".format(rec))
# print("AUC: {}".format(AUC))
# print("Time consumed for training: %4.3f seconds" % (knn_train_time))
# print("Time consumed for prediction: %6.5f seconds" % (knn_prediction_time))

In [7]:
import time

In [8]:
training_start = time.perf_counter()
xgbmodel=ran_search_xgb.fit(trainX,trainY)
training_end = time.perf_counter()

train_time = training_end-training_start

print("Time consumed for training: %4.3f seconds" % (train_time))

Time consumed for training: 5546.267 seconds


In [9]:
print("Best parameters found: ", ran_search_xgb.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(ran_search_xgb.best_score_)))

Best parameters found:  {'subsample': 1, 'n_estimators': 3000, 'max_depth': 5, 'learning_rate': 1, 'colsample_bytree': 0.9}
Lowest RMSE found:  0.4290928319887428


In [15]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score, roc_curve

In [12]:
df2=pd.read_csv('../data/le_test.csv',index_col=0)

In [20]:
testX=df2.drop('readmitted',axis=1).copy()
testY=df2['readmitted']

In [23]:
prediction_start = time.perf_counter()
preds = xgbmodel.predict(testX)
prediction_end = time.perf_counter()

prediction_time = prediction_end-prediction_start

print("Time consumed for prediction: %6.5f seconds" % (prediction_time))

Time consumed for prediction: 5.97720 seconds


In [26]:
preds[:100]

array([ 0.11785704, -0.08038265,  0.02597851,  0.03398538, -0.3585565 ,
        0.38043416, -0.04759473,  0.0736424 ,  0.11565429,  0.2569903 ,
       -0.25421864,  0.36298358,  0.28497243,  0.11373284,  0.18197161,
        0.19930771,  0.23657379, -0.3562013 , -0.13396782,  0.19981071,
        0.29282403,  0.19617486, -0.12990159,  0.13333666, -0.28703928,
       -0.05574435,  0.0114353 ,  0.24104747,  0.2934407 , -0.1173526 ,
        0.37156057,  0.5073163 ,  0.15572852, -0.03937083, -0.34466344,
       -0.19455868, -0.45188767,  0.30009776, -0.03502268, -0.3298875 ,
        0.93974745, -0.12837565,  0.28789312, -0.09214598,  0.26599926,
       -0.6353445 ,  0.16131642, -0.28260082, -0.85868454,  0.23840272,
       -0.13535744,  0.02031106, -0.18093985,  0.03039771,  0.12564167,
        0.34560812, -0.42508912, -0.6581745 ,  0.51037353,  0.4846528 ,
        0.37863812, -0.46838295,  0.09974626,  0.53806067, -0.062819  ,
        0.3454277 ,  0.49358085,  0.38896495, -0.15476197,  0.04

In [ ]:
# acc = accuracy_score(testY, preds)
# pre = precision_score(testY, preds)
# rec = recall_score(testY, preds)
# AUC = roc_auc_score(testY, preds)

                    
# print("Accuracy: {:.4%}".format(acc))
# print("Precision: {:.4%}".format(pre))
# print("Recall: {:.4%}".format(rec))
# print("AUC: {}".format(AUC))